# RAG DB

---

This is a demonstation notebook showing how to create the a LLM agent that will interact with a Database. With a URI it is possible to connect to practically all SQL databases. Here we simply work with pythons SQLite3, where we load a database from CSV > Pandas > SQlite.

It is worth noting that getting basic functionality from Langchain is a formality, but the chain is quite complex. Ideally our final system with have lots of failsafe's and will dictate which tools and types of queries we want the LLM to use. It is here that I have encountered problems implementing more advanced functionality, and the final system did not act as required. No doubt workarounds can be found, but it feels like this Langchain functionality is still WIP.

---

## $\color{blue}{Sections:}$
* Admin
* Data
* Tests

---
## $\color{blue}{Admin}$
---

In [ ]:
%%capture
pip install langchain-openai

In [ ]:
%%capture
!pip install langchain

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI

from google.colab import drive
import sqlite3
import pandas as pd
import os

In [ ]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive'

Mounted at /content/drive
/content/drive/MyDrive


---
## $\color{blue}{Data}$
---
For the purposes of demonstration, we load a CSV into pandas DataFrame.
The DataFrame is used to create a database with SQLite. This database is then wrapped by the Langchain wrapper.

The Langchain wrapper builds from a URI. In theory we can connect to any database with the URI.

---
#### $\color{red}{DataFrame}$
---

The data is an extended version of the classic Titanic dataset. There may be some ambiguity in the column names, eg. Sibsp refers to the siblings and spouses. This is the type of information the may confuse the LLM, and there may be some work to do to make the LLM understand the data.

In [ ]:
df = pd.read_csv('RAG_tutorial/Data/full.csv')

In [ ]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'WikiId', 'Name_wiki',
       'Age_wiki', 'Hometown', 'Boarded', 'Destination', 'Lifeboat', 'Body',
       'Class'],
      dtype='object')

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,WikiId,Name_wiki,Age_wiki,Hometown,Boarded,Destination,Lifeboat,Body,Class
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,S,691.0,"Braund, Mr. Owen Harris",22.0,"Bridgerule, Devon, England",Southampton,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,C,90.0,"Cumings, Mrs. Florence Briggs (née Thayer)",35.0,"New York, New York, US",Cherbourg,"New York, New York, US",4,NaN,1.0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,S,865.0,"Heikkinen, Miss Laina",26.0,"Jyväskylä, Finland",Southampton,New York City,14?,NaN,3.0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,S,127.0,"Futrelle, Mrs. Lily May (née Peel)",35.0,"Scituate, Massachusetts, US",Southampton,"Scituate, Massachusetts, US",D,NaN,1.0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,S,627.0,"Allen, Mr. William Henry",35.0,"Birmingham, West Midlands, England",Southampton,New York City,NaN,NaN,3.0


---
#### $\color{red}{Database}$
---

Now we have the DataFrame, we can quickly transform it into a database.

In [ ]:
def dataframe_to_sqlite(df, db_name, table_name):
    """
    Converts a Pandas DataFrame to an SQLite database table.

    Args:
        df (pd.DataFrame): The DataFrame to be converted.
        db_name (str): Name of the SQLite database file.
        table_name (str): Name of the table to be created in the database.

    Returns:
        None
    """
    try:
        # Create an SQLite connection
        conn = sqlite3.connect(db_name)

        # Write the DataFrame to the specified table
        df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)

        print(f"DataFrame successfully written to SQLite table '{table_name}' in '{db_name}'.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Close the connection
        conn.close()

In [ ]:
db_name = 'titanic.db'
db_table = 'titanic_table'
uri = "sqlite:///titanic.db"

In [ ]:
dataframe_to_sqlite(df, db_name, db_table)

DataFrame successfully written to SQLite table 'titanic_table' in 'titanic.db'.


---
We can check that the underlying database is functioning.
---
---

In [ ]:
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
cursor.execute(f'SELECT * FROM {db_table};')
rows = cursor.fetchall()
for row in rows[:10]:
    print(row)

(1, 0.0, 3, 'Braund, Mr. Owen Harris', 'male', 22.0, 1, 0, 'A/5 21171', 7.25, None, 'S', 691.0, 'Braund, Mr. Owen Harris', 22.0, 'Bridgerule, Devon, England', 'Southampton', "Qu'Appelle Valley, Saskatchewan, Canada", None, None, 3.0)
(2, 1.0, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', 38.0, 1, 0, 'PC 17599', 71.2833, 'C85', 'C', 90.0, 'Cumings, Mrs. Florence Briggs (née Thayer)', 35.0, 'New York, New York, US', 'Cherbourg', 'New York, New York, US', '4', None, 1.0)
(3, 1.0, 3, 'Heikkinen, Miss. Laina', 'female', 26.0, 0, 0, 'STON/O2. 3101282', 7.925, None, 'S', 865.0, 'Heikkinen, Miss Laina', 26.0, 'Jyväskylä, Finland', 'Southampton', 'New York City', '14?', None, 3.0)
(4, 1.0, 1, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'female', 35.0, 1, 0, '113803', 53.1, 'C123', 'S', 127.0, 'Futrelle, Mrs. Lily May (née Peel)', 35.0, 'Scituate, Massachusetts, US', 'Southampton', 'Scituate, Massachusetts, US', 'D', None, 1.0)
(5, 0.0, 3, 'Allen, Mr. William Henry', 'm

---
#### $\color{red}{Langchain-Database}$
---

Now we wrap the database with the Langchain object.

In [ ]:
db = SQLDatabase.from_uri(uri)

In [ ]:
print(db.get_table_info())


CREATE TABLE titanic_table (
	"PassengerId" INTEGER, 
	"Survived" REAL, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT, 
	"WikiId" REAL, 
	"Name_wiki" TEXT, 
	"Age_wiki" REAL, 
	"Hometown" TEXT, 
	"Boarded" TEXT, 
	"Destination" TEXT, 
	"Lifeboat" TEXT, 
	"Body" TEXT, 
	"Class" REAL
)

/*
3 rows from titanic_table table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked	WikiId	Name_wiki	Age_wiki	Hometown	Boarded	Destination	Lifeboat	Body	Class
1	0.0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S	691.0	Braund, Mr. Owen Harris	22.0	Bridgerule, Devon, England	Southampton	Qu'Appelle Valley, Saskatchewan, Canada	None	None	3.0
2	1.0	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C	90.0	Cumings, Mrs. Florence Briggs (née Thayer)	35.0	New York, New York, US	Cherbourg	New York, New York, 

---
We will be using GPT-3.5 for the demo, so drop in an OpenAI key
---
---

In [ ]:
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Open AI API Key:")

Open AI API Key:··········


---
The toolkit specifies the tools that are available to the agent, namely the databse, and the LLM
---
---

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


---
The agent executor wraps all the functionality required including the agent type defines how the logic of the agent functions.
---
---

In [ ]:
agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
)

---
## $\color{blue}{Test}$
---

Very quickly if we have a database and an LLM we can interact with db in natural language.


Here the LLM is given a modified table name, but manages to infer the table that we are referring to


In [ ]:
agent_executor.invoke("what columns are in the titanic table")



> Entering new SQL Agent Executor chain...
I should use the sql_db_schema tool to get the schema of the titanic table
Action: sql_db_schema
Action Input: titanicError: table_names {'titanic'} not found in databaseI should first check the list of tables in the database to make sure the titanic table exists
Action: sql_db_list_tables
Action Input: titanic_tableNow that I know the titanic table exists, I can use the sql_db_schema tool to get the schema of the titanic table
Action: sql_db_schema
Action Input: titanicError: table_names {'titanic'} not found in databaseI should double check the table name and try again
Action: sql_db_schema
Action Input: titanic_table
CREATE TABLE titanic_table (
	"PassengerId" INTEGER, 
	"Survived" REAL, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT, 
	"WikiId" REAL, 
	"Name_wiki" TEXT, 
	"Age_wiki" REAL, 
	"Hometown" TEXT, 
	"Boarded" 

{'input': 'what columns are in the titanic table',
 'output': 'PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, WikiId, Name_wiki, Age_wiki, Hometown, Boarded, Destination, Lifeboat, Body, Class'}

---
**Here on a more complex query we see the logic of the agent in operation.**

```
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''
```

We see the format of the prompt goes through numerous steps using the tools, before the LLM is satisfied that it has the correct answer.

The following example starts with an incorrect SQL statement where the agent tries to use a table that doesn't exist. After capturing the error the agent decides to search the schema of the tables available.

The agent understands that the titanic_table 'name' field might lead to the correct answer and makes a new query. The query returns the correct answer, and the agent responds to the initial request, satisfied that it has found the correct information.

In [ ]:
agent_executor.invoke("Was there anybody from the Futrelle family on board the titanic")



> Entering new SQL Agent Executor chain...
I need to check the database to see if there were any passengers from the Futrelle family on board the Titanic.
Action: sql_db_query_checker
Action Input: SELECT * FROM passengers WHERE last_name = 'Futrelle' SELECT * FROM passengers WHERE last_name = 'Futrelle'I should now execute the query to get the final answer.
Action: sql_db_query
Action Input: SELECT * FROM passengers WHERE last_name = 'Futrelle'Error: (sqlite3.OperationalError) no such table: passengers
[SQL: SELECT * FROM passengers WHERE last_name = 'Futrelle']
(Background on this error at: https://sqlalche.me/e/20/e3q8)I need to first check the list of tables in the database to ensure the 'passengers' table exists.
Action: sql_db_list_tables
Action Input: titanic_tableI should now check the schema of the 'titanic_table' to see if it contains information about passengers.
Action: sql_db_schema
Action Input: titanic_table
CREATE TABLE titanic_table (
	"PassengerId" INTEGER, 
	"Survi

{'input': 'Was there anybody from the Futrelle family on board the titanic',
 'output': 'Yes, there were passengers from the Futrelle family on board the Titanic.'}